# Bell Inequality: Testing Quantum Nonlocality

## Introduction

The Bell inequality, formulated by physicist John Stewart Bell in 1964, provides a means to experimentally distinguish between quantum mechanics and local hidden variable theories. This inequality represents one of the most profound results in the foundations of quantum physics, demonstrating that no local realistic theory can reproduce all predictions of quantum mechanics.

## Theoretical Background

### Einstein-Podolsky-Rosen (EPR) Paradox

In 1935, Einstein, Podolsky, and Rosen argued that quantum mechanics was incomplete. They proposed that "hidden variables" must exist to explain quantum correlations without invoking nonlocality.

### Bell's Theorem

Bell showed that any local hidden variable theory must satisfy certain statistical inequalities. Consider two entangled particles measured by observers Alice and Bob with detector settings $\mathbf{a}$, $\mathbf{a}'$ and $\mathbf{b}$, $\mathbf{b}'$ respectively.

For measurement outcomes $A(\mathbf{a}, \lambda) = \pm 1$ and $B(\mathbf{b}, \lambda) = \pm 1$, where $\lambda$ represents hidden variables, the correlation function is:

$$E(\mathbf{a}, \mathbf{b}) = \int d\lambda \, \rho(\lambda) \, A(\mathbf{a}, \lambda) \, B(\mathbf{b}, \lambda)$$

### CHSH Inequality

The Clauser-Horne-Shimony-Holt (CHSH) form of Bell's inequality states:

$$|S| = |E(\mathbf{a}, \mathbf{b}) - E(\mathbf{a}, \mathbf{b}') + E(\mathbf{a}', \mathbf{b}) + E(\mathbf{a}', \mathbf{b}')| \leq 2$$

This bound of 2 applies to all local hidden variable theories.

### Quantum Mechanical Prediction

For the singlet state:

$$|\psi^-\rangle = \frac{1}{\sqrt{2}}(|\uparrow\downarrow\rangle - |\downarrow\uparrow\rangle)$$

The quantum correlation function is:

$$E_{QM}(\mathbf{a}, \mathbf{b}) = -\cos(\theta_{ab})$$

where $\theta_{ab}$ is the angle between measurement directions.

For optimal angles $\mathbf{a} = 0°$, $\mathbf{a}' = 90°$, $\mathbf{b} = 45°$, $\mathbf{b}' = 135°$:

$$S_{QM} = 2\sqrt{2} \approx 2.828$$

This exceeds the classical bound of 2, demonstrating quantum nonlocality.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Set random seed for reproducibility
np.random.seed(42)

# Physical constants and parameters
N_TRIALS = 10000  # Number of measurement trials

## Quantum Mechanical Simulation

We simulate measurements on an entangled singlet state. For each trial, we:
1. Generate an entangled pair in the singlet state
2. Measure spin projections along chosen axes
3. Record correlated outcomes

In [ ]:
def quantum_correlation(theta_a, theta_b, n_trials=N_TRIALS):
    """
    Simulate quantum correlation for singlet state measurements.
    
    For the singlet state |ψ⁻⟩, the probability of outcomes is:
    P(+,+) = P(-,-) = (1/2)sin²(θ/2)
    P(+,-) = P(-,+) = (1/2)cos²(θ/2)
    
    where θ = θ_a - θ_b is the relative angle.
    
    Parameters:
    -----------
    theta_a : float
        Alice's measurement angle in radians
    theta_b : float
        Bob's measurement angle in radians
    n_trials : int
        Number of measurement trials
    
    Returns:
    --------
    correlation : float
        Measured correlation E(a,b)
    outcomes_a : array
        Alice's measurement outcomes (+1 or -1)
    outcomes_b : array
        Bob's measurement outcomes (+1 or -1)
    """
    theta_diff = theta_a - theta_b
    
    # Probability of same outcomes (++, --)
    p_same = np.sin(theta_diff / 2)**2
    
    # Generate random outcomes based on quantum probabilities
    random_vals = np.random.random(n_trials)
    
    # Alice's outcomes: randomly +1 or -1 with equal probability
    outcomes_a = np.where(np.random.random(n_trials) < 0.5, 1, -1)
    
    # Bob's outcomes: correlated with Alice according to quantum mechanics
    # If same outcome (with prob p_same), multiply by 1; otherwise by -1
    same_outcome = random_vals < p_same
    outcomes_b = np.where(same_outcome, outcomes_a, -outcomes_a)
    
    # Calculate correlation
    correlation = np.mean(outcomes_a * outcomes_b)
    
    return correlation, outcomes_a, outcomes_b


def theoretical_quantum_correlation(theta_a, theta_b):
    """
    Theoretical quantum correlation for singlet state.
    E(a,b) = -cos(θ_a - θ_b)
    """
    return -np.cos(theta_a - theta_b)

## Local Hidden Variable Model

We implement a classical local hidden variable model where each particle carries a predetermined "instruction set" $\lambda$ that determines measurement outcomes.

In [ ]:
def local_hidden_variable_correlation(theta_a, theta_b, n_trials=N_TRIALS):
    """
    Simulate a local hidden variable model.
    
    Each particle pair carries a hidden variable λ (uniformly distributed angle).
    Measurement outcome is +1 if particle's hidden angle is within π/2 of 
    detector angle, -1 otherwise.
    
    This represents the "best" classical strategy that mimics quantum correlations.
    """
    # Hidden variable: random angle for each particle pair
    lambda_vals = np.random.uniform(0, 2*np.pi, n_trials)
    
    # Alice's outcome based on her detector angle
    diff_a = np.abs(np.mod(lambda_vals - theta_a + np.pi, 2*np.pi) - np.pi)
    outcomes_a = np.where(diff_a < np.pi/2, 1, -1)
    
    # Bob's outcome: anticorrelated hidden variable (λ + π)
    lambda_b = lambda_vals + np.pi
    diff_b = np.abs(np.mod(lambda_b - theta_b + np.pi, 2*np.pi) - np.pi)
    outcomes_b = np.where(diff_b < np.pi/2, 1, -1)
    
    correlation = np.mean(outcomes_a * outcomes_b)
    
    return correlation, outcomes_a, outcomes_b


def theoretical_lhv_correlation(theta):
    """
    Theoretical correlation for the optimal local hidden variable model.
    This gives a triangular correlation function.
    """
    theta = np.mod(theta, 2*np.pi)
    return np.where(theta <= np.pi, 
                    1 - 2*theta/np.pi, 
                    -3 + 2*theta/np.pi)

## CHSH Test Implementation

We now compute the CHSH parameter $S$ for both models using optimal measurement angles.

In [ ]:
def compute_chsh_parameter(correlation_func, a, a_prime, b, b_prime, n_trials=N_TRIALS):
    """
    Compute the CHSH parameter S.
    
    S = E(a,b) - E(a,b') + E(a',b) + E(a',b')
    
    Classical bound: |S| ≤ 2
    Quantum maximum: |S| = 2√2 ≈ 2.828
    """
    E_ab, _, _ = correlation_func(a, b, n_trials)
    E_ab_prime, _, _ = correlation_func(a, b_prime, n_trials)
    E_a_prime_b, _, _ = correlation_func(a_prime, b, n_trials)
    E_a_prime_b_prime, _, _ = correlation_func(a_prime, b_prime, n_trials)
    
    S = E_ab - E_ab_prime + E_a_prime_b + E_a_prime_b_prime
    
    return S, [E_ab, E_ab_prime, E_a_prime_b, E_a_prime_b_prime]


# Optimal angles for maximum CHSH violation
# a = 0°, a' = 90°, b = 45°, b' = 135°
a = 0
a_prime = np.pi/2
b = np.pi/4
b_prime = 3*np.pi/4

print("CHSH Test Results")
print("=" * 50)
print(f"\nMeasurement angles:")
print(f"  Alice: a = {np.degrees(a):.1f}°, a' = {np.degrees(a_prime):.1f}°")
print(f"  Bob:   b = {np.degrees(b):.1f}°, b' = {np.degrees(b_prime):.1f}°")
print(f"\nNumber of trials: {N_TRIALS:,}")

# Quantum mechanical results
S_qm, correlations_qm = compute_chsh_parameter(
    quantum_correlation, a, a_prime, b, b_prime
)
print(f"\nQuantum Mechanics:")
print(f"  E(a,b) = {correlations_qm[0]:.4f}")
print(f"  E(a,b') = {correlations_qm[1]:.4f}")
print(f"  E(a',b) = {correlations_qm[2]:.4f}")
print(f"  E(a',b') = {correlations_qm[3]:.4f}")
print(f"  S = {S_qm:.4f}")
print(f"  Theoretical S = 2√2 = {2*np.sqrt(2):.4f}")

# Local hidden variable results
S_lhv, correlations_lhv = compute_chsh_parameter(
    local_hidden_variable_correlation, a, a_prime, b, b_prime
)
print(f"\nLocal Hidden Variables:")
print(f"  E(a,b) = {correlations_lhv[0]:.4f}")
print(f"  E(a,b') = {correlations_lhv[1]:.4f}")
print(f"  E(a',b) = {correlations_lhv[2]:.4f}")
print(f"  E(a',b') = {correlations_lhv[3]:.4f}")
print(f"  S = {S_lhv:.4f}")
print(f"  Classical bound: |S| ≤ 2")

print(f"\nBell Inequality Violation: {'YES' if abs(S_qm) > 2 else 'NO'}")

## Visualization

We create comprehensive visualizations comparing quantum and classical correlations.

In [ ]:
# Create figure with multiple subplots
fig = plt.figure(figsize=(14, 10))

# Plot 1: Correlation functions vs. relative angle
ax1 = fig.add_subplot(2, 2, 1)

theta_range = np.linspace(0, 2*np.pi, 200)

# Theoretical curves
qm_theory = -np.cos(theta_range)
lhv_theory = theoretical_lhv_correlation(theta_range)

# Simulated data points
theta_samples = np.linspace(0, 2*np.pi, 20)
qm_sim = [quantum_correlation(0, theta, 5000)[0] for theta in theta_samples]
lhv_sim = [local_hidden_variable_correlation(0, theta, 5000)[0] for theta in theta_samples]

ax1.plot(np.degrees(theta_range), qm_theory, 'b-', linewidth=2, label='QM Theory')
ax1.plot(np.degrees(theta_range), lhv_theory, 'r--', linewidth=2, label='LHV Theory')
ax1.scatter(np.degrees(theta_samples), qm_sim, c='blue', s=50, alpha=0.7, label='QM Simulation')
ax1.scatter(np.degrees(theta_samples), lhv_sim, c='red', s=50, alpha=0.7, marker='s', label='LHV Simulation')

ax1.set_xlabel('Relative Angle θ (degrees)', fontsize=12)
ax1.set_ylabel('Correlation E(θ)', fontsize=12)
ax1.set_title('Correlation Functions: Quantum vs Classical', fontsize=14)
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 360)
ax1.set_ylim(-1.1, 1.1)

# Plot 2: CHSH parameter distribution
ax2 = fig.add_subplot(2, 2, 2)

# Run multiple experiments
n_experiments = 500
S_qm_dist = []
S_lhv_dist = []

for _ in range(n_experiments):
    s_qm, _ = compute_chsh_parameter(quantum_correlation, a, a_prime, b, b_prime, 1000)
    s_lhv, _ = compute_chsh_parameter(local_hidden_variable_correlation, a, a_prime, b, b_prime, 1000)
    S_qm_dist.append(s_qm)
    S_lhv_dist.append(s_lhv)

ax2.hist(S_qm_dist, bins=30, alpha=0.7, color='blue', label=f'QM (μ={np.mean(S_qm_dist):.3f})', density=True)
ax2.hist(S_lhv_dist, bins=30, alpha=0.7, color='red', label=f'LHV (μ={np.mean(S_lhv_dist):.3f})', density=True)
ax2.axvline(x=2, color='black', linestyle='--', linewidth=2, label='Classical bound')
ax2.axvline(x=2*np.sqrt(2), color='green', linestyle=':', linewidth=2, label='QM maximum')

ax2.set_xlabel('CHSH Parameter S', fontsize=12)
ax2.set_ylabel('Probability Density', fontsize=12)
ax2.set_title('CHSH Parameter Distribution', fontsize=14)
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

# Plot 3: Statistical significance
ax3 = fig.add_subplot(2, 2, 3)

# Calculate S vs number of trials
trial_counts = np.logspace(2, 4, 15).astype(int)
S_means = []
S_stds = []

for n in trial_counts:
    s_values = [compute_chsh_parameter(quantum_correlation, a, a_prime, b, b_prime, n)[0] 
                for _ in range(50)]
    S_means.append(np.mean(s_values))
    S_stds.append(np.std(s_values))

S_means = np.array(S_means)
S_stds = np.array(S_stds)

ax3.errorbar(trial_counts, S_means, yerr=2*S_stds, fmt='o-', color='blue', 
             capsize=3, label='QM simulation (2σ)')
ax3.axhline(y=2*np.sqrt(2), color='green', linestyle='--', linewidth=2, label='QM theory')
ax3.axhline(y=2, color='red', linestyle=':', linewidth=2, label='Classical bound')

ax3.set_xscale('log')
ax3.set_xlabel('Number of Trials', fontsize=12)
ax3.set_ylabel('CHSH Parameter S', fontsize=12)
ax3.set_title('Convergence to Quantum Prediction', fontsize=14)
ax3.legend(loc='lower right')
ax3.grid(True, alpha=0.3)

# Plot 4: Violation strength as function of angle settings
ax4 = fig.add_subplot(2, 2, 4)

# Vary the angle between a and a'
delta_angles = np.linspace(0, np.pi, 50)
S_values = []

for delta in delta_angles:
    # Keep optimal relative configuration
    test_a = 0
    test_a_prime = delta
    test_b = delta/2
    test_b_prime = delta/2 + np.pi/2
    
    # Theoretical quantum value
    E1 = -np.cos(test_a - test_b)
    E2 = -np.cos(test_a - test_b_prime)
    E3 = -np.cos(test_a_prime - test_b)
    E4 = -np.cos(test_a_prime - test_b_prime)
    S_values.append(E1 - E2 + E3 + E4)

ax4.plot(np.degrees(delta_angles), S_values, 'b-', linewidth=2)
ax4.fill_between(np.degrees(delta_angles), 2, S_values, 
                  where=np.array(S_values) > 2, alpha=0.3, color='blue', 
                  label='Bell violation region')
ax4.axhline(y=2, color='red', linestyle='--', linewidth=2, label='Classical bound')
ax4.axhline(y=2*np.sqrt(2), color='green', linestyle=':', linewidth=2, label='Maximum violation')
ax4.axvline(x=90, color='gray', linestyle=':', alpha=0.5)

ax4.set_xlabel("Angle Separation Δ = a' - a (degrees)", fontsize=12)
ax4.set_ylabel('CHSH Parameter S', fontsize=12)
ax4.set_title('CHSH Value vs Angle Configuration', fontsize=14)
ax4.legend(loc='upper left')
ax4.grid(True, alpha=0.3)
ax4.set_xlim(0, 180)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to 'plot.png'")

## Statistical Analysis

We perform a rigorous statistical test of the Bell inequality violation.

In [ ]:
# Perform statistical hypothesis test
print("Statistical Hypothesis Test")
print("=" * 50)
print("\nNull hypothesis H₀: |S| ≤ 2 (local realism holds)")
print("Alternative H₁: |S| > 2 (quantum nonlocality)\n")

# Large sample for precise estimate
n_large = 100000
S_final, correlations_final = compute_chsh_parameter(
    quantum_correlation, a, a_prime, b, b_prime, n_large
)

# Bootstrap for confidence interval
n_bootstrap = 1000
bootstrap_S = []
for _ in range(n_bootstrap):
    s, _ = compute_chsh_parameter(quantum_correlation, a, a_prime, b, b_prime, n_large // 10)
    bootstrap_S.append(s)

ci_lower = np.percentile(bootstrap_S, 2.5)
ci_upper = np.percentile(bootstrap_S, 97.5)
std_error = np.std(bootstrap_S)

# Z-score for testing against classical bound
z_score = (np.mean(bootstrap_S) - 2) / std_error
p_value = 1 - stats.norm.cdf(z_score)

print(f"Sample size: {n_large:,}")
print(f"Measured S: {S_final:.4f}")
print(f"95% Confidence Interval: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"Standard Error: {std_error:.4f}")
print(f"\nZ-score (against S=2): {z_score:.2f}")
print(f"P-value: {p_value:.2e}")
print(f"\nConclusion: {'Reject H₀' if p_value < 0.05 else 'Fail to reject H₀'} at α=0.05")
print(f"\nThe quantum mechanical simulation shows a statistically significant")
print(f"violation of the Bell inequality, with S = {S_final:.3f} > 2.")

## Conclusions

This simulation demonstrates several key aspects of Bell's inequality:

1. **Quantum correlations exceed classical bounds**: The CHSH parameter $S \approx 2\sqrt{2}$ for quantum mechanics, violating the classical limit of $|S| \leq 2$.

2. **Optimal angles**: Maximum violation occurs at specific angles: $\mathbf{a} = 0°$, $\mathbf{a}' = 90°$, $\mathbf{b} = 45°$, $\mathbf{b}' = 135°$.

3. **Statistical significance**: With sufficient measurements, the violation is highly statistically significant.

4. **Implications**: The violation of Bell's inequality rules out local hidden variable theories, confirming quantum entanglement as a genuine physical phenomenon.

### Historical Note

The first experimental tests by Freedman and Clauser (1972) and Aspect et al. (1982) confirmed these predictions. The 2022 Nobel Prize in Physics was awarded to Aspect, Clauser, and Zeilinger for their experiments establishing the violation of Bell inequalities and pioneering quantum information science.